In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display
from pydantic import BaseModel

import os, json
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

In [ ]:
load_dotenv(override=True)
# openai = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="ollama")
openai = OpenAI()

In [ ]:
def send_email(subject: str, html_content: str):
    """
    """
    message = Mail(
        from_email='bhattacharyyaprasun47@gmail.com',
        to_emails='bhattacharyyaprasun47+test@gmail.com',
        subject=subject,
        html_content=html_content)
    try:
        sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
        # sg.set_sendgrid_data_residency("eu")
        # uncomment the above line if you are sending mail using a regional EU subuser
        response = sg.send(message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as e:
        print(e.message)

def notify_user_details(email: str, 
    name: str = "Name not provided", notes: str = "Notes not provided"):
    send_email("New contact!", f"<strong>Someone<strong> trying to contact with you! Name: {name}, Email: {email}, Notes: {notes}")
    return {"notified": "ok"}

notify_user_details_json = {
    "name": "notify_user_details",
    "description": "Use this tool to notify that a user is trying to contact and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of the user"
            },
            "name": {
                "type": "string",
                "description": "The name of the user, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional notes about the conversation that worth's notifying to get more context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

TOOLS = [
    {
        "type": "function",
        "function": notify_user_details_json
    }
]

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results

In [ ]:
reader = PdfReader("resources/linkedin.pdf")
LINKEDIN = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        LINKEDIN += text

In [ ]:
with open("resources/summary.txt", "r", encoding="utf-8") as f:
    SUMMARY = f.read()

In [ ]:
NAME = "Prasun Bhattacharyya"

In [ ]:
SYSTEM_PROMPT = f"You are acting as {NAME}. You are answering questions on {NAME}'s website, \
particularly questions related to {NAME}'s career, background, skills and experience. \
Your responsibility is to represent {NAME} for interactions on the website as faithfully as possible. \
You are given a summary of {NAME}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so. \
If the user is trying to connect with me ask them to provide their email address and name and notify using notify_user_details tool."

SYSTEM_PROMPT += f"\n\n## Summary:\n{SUMMARY}\n\n## LinkedIn Profile:\n{LINKEDIN}\n\n"
SYSTEM_PROMPT += f"With this context, please chat with the user, always staying in character as {NAME}."


In [ ]:
# Create a Pydantic model for the Evaluation
class Evaluation(BaseModel):
    """
    A Pydantic model for the evaluation of a response to a question.
    Args:
        is_acceptable (bool): Whether the response is acceptable.
        feedback (str): The feedback on the response.
    """
    is_acceptable: bool
    feedback: str


In [ ]:
EVALUATOR_SYSTEM_PROMPT = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {NAME} and is representing {NAME} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {NAME} in the form of their summary and LinkedIn details. Here's the information:"

EVALUATOR_SYSTEM_PROMPT += f"\n\n## Summary:\n{SUMMARY}\n\n## LinkedIn Profile:\n{LINKEDIN}\n\n"
EVALUATOR_SYSTEM_PROMPT += f"Additionally, if the user is trying to connect or get in touch, this Agent will ask for name and email address and use notify_user_details tool to notify.\n\n"
EVALUATOR_SYSTEM_PROMPT += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply: str, message: str, history: list) -> str:
    """
    A function to create a user prompt for the evaluator.
    Args:
        reply (str): The latest response from the Agent.
        message (str): The latest message from the User.
        history (list): The history of the conversation between the User and the Agent.
    Returns:
        str: The user prompt for the evaluator.
    """
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."

    print(user_prompt)
    
    return user_prompt

In [ ]:
def evaluate(reply: str, message: str, history: list) -> Evaluation:
    """
    Evaluate the response to a question.
    Args:
        reply (str): The latest response from the Agent.
        message (str): The latest message from the User.
        history (list): The history of the conversation between the User and the Agent.
    Returns:
        Evaluation: The evaluation of the response.
    """

    messages = [{"role": "system", "content": EVALUATOR_SYSTEM_PROMPT}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
def rerun(reply: str, message: str, history: list, feedback: str) -> str:
    """
    Rerun the conversation with the updated system prompt.
    Args:
        reply (str): The latest response from the Agent.
        message (str): The latest message from the User.
        history (list): The history of the conversation between the User and the Agent.
        feedback (str): The feedback on the response.
    Returns:
        str: The response from the Agent.
    """

    updated_system_prompt = SYSTEM_PROMPT + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message: str, history: list) -> str:
    """
    Chat with the agent.
    Args:
        message (str): The message to chat with the agent.
        history (list): The history of the conversation.
    Returns:
        str: The response from the agent.
    """
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=TOOLS)
        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message_obj = response.choices[0].message
            tool_calls = message_obj.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message_obj)
            messages.extend(results)
        else:
            done = True

    reply = response.choices[0].message.content
    
    evaluation = evaluate(reply, message, history)
    
    while not evaluation.is_acceptable:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
        evaluation = evaluate(reply, message, history)
    return reply
    

In [ ]:
# Create a chat interface
gr.ChatInterface(chat, type="messages").launch()